In [ ]:
import linpde_gp
import numpy as np
import probnum as pn
from gp_fvm.plot.dim1 import plot_function, compare_to_solution_gp

pn.config.default_solver_linpde_gp = linpde_gp.solvers.CholeskySolver(dense=True)

# Problem

In [ ]:
spatial_domain = linpde_gp.domains.asdomain([-1.0, 1.0])
temporal_domain = linpde_gp.domains.asdomain([0., 5.])
alpha = 0.1
coefficients = [1.0, 0.5, 0.25]

problem = linpde_gp.problems.pde.HeatEquationDirichletProblem(
    t0=temporal_domain[0],
    T=temporal_domain[1],
    spatial_domain=spatial_domain,
    alpha=alpha,
    initial_values=linpde_gp.functions.TruncatedSineSeries(
        spatial_domain,
        coefficients=coefficients,
    ),
)

box_domain = linpde_gp.domains.Box([problem.domain[0], problem.domain[1]])

# Solution

In [ ]:
plot_function(problem.solution, problem.domain, ylims=[0., 1.45])

# Prior

In [ ]:
lengthscale_t = 2.5
lengthscale_x = 2.0
output_scale = 1.0

N_ic = 10
N_bc = 50

In [ ]:
def get_prior(l_t, l_x, output_scale):
    return pn.randprocs.GaussianProcess(
        mean=linpde_gp.functions.Zero(input_shape=(2,)),
        cov=output_scale**2
        * linpde_gp.randprocs.covfuncs.TensorProduct(
            linpde_gp.randprocs.covfuncs.Matern((), nu=1.5, lengthscales=l_t),
            linpde_gp.randprocs.covfuncs.Matern((), nu=2.5, lengthscales=l_x),
        ),
    )

u_prior = get_prior(lengthscale_t, lengthscale_x, output_scale)

# Initial condition

In [ ]:
def condition_ic(prior):
    X_ic = problem.initial_domain.uniform_grid(N_ic, inset=1e-6)
    Y_ic = problem.initial_condition.values(X_ic[..., 1])

    return prior.condition_on_observations(Y_ic, X_ic)

In [ ]:
u_ic = condition_ic(u_prior)
fig, _ = compare_to_solution_gp(u_ic, problem.solution, problem.domain, label="$u_{\\text{ic}}$")

# Boundary conditions

In [ ]:
def condition_bc(prior):
    u_ic_bc = prior
    for bc in problem.boundary_conditions:
        X_bc = bc.boundary.uniform_grid(N_bc)
        Y_bc = bc.values(X_bc)

        u_ic_bc = u_ic_bc.condition_on_observations(Y_bc, X=X_bc)
    return u_ic_bc

In [ ]:
u_ic_bc = condition_bc(u_ic)

In [ ]:
fig, _ = compare_to_solution_gp(u_ic_bc, problem.solution, problem.domain, label="$u_{\\text{ic, bc}}$", num_t=3)

# Finite volumes

## Utility

In [ ]:
from gp_fvm.finite_volumes import get_grid_from_depth

def condition_fv(prior, depth):
    fv_domains = get_grid_from_depth(box_domain, depth)
    fv = linpde_gp.linfunctls.FiniteVolumeFunctional(fv_domains, problem.pde.diffop)
    return prior.condition_on_observations(L=fv, Y=np.zeros(fv_domains.shape))

In [ ]:
u_fv = condition_fv(u_ic_bc, 3)

In [ ]:
fig, _ = compare_to_solution_gp(u_fv, problem.solution, problem.domain, label='$u_{\\text{FV}}$', ylims=[0., 0.8], skip_t0=True)